In [3]:
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib as mpl
#mpl.rcParams['axes.linewidth'] = 2
#import pandas as pd
#import math 
import os
import pickle
#import scipy.ndimage as ndimage
#from matplotlib.ticker import AutoMinorLocator
import json
import sys

#project_folder = "C:\\Users\\giogu\\Documents\\Acustica\\python_routines" 
#sys.path.append(project_folder)

sys.path.append("../../")

from pyAcousticsAnalysis.instruments.LD_831.read_data import read_LD_831_data
from pyAcousticsAnalysis.instruments.commons import read_preprocessed_data#, get_masks
from pyAcousticsAnalysis.utils.helper_functions import get_mean_levels, get_min_levels, generate_freq_third_octaves, logarithmic_average
#from pyEnvAcoustics.plotting.plot_functions import plot_mean_Leq, get_freq_x_ticks
from pyAcousticsAnalysis.plotting.plot_SPL import plot_SPL
from pyAcousticsAnalysis.plotting.plot_time_history import plot_time_history
from pyAcousticsAnalysis.plotting.plot_min_SPL import plot_min_SPL
from pyAcousticsAnalysis.plotting.helper_functions import calculate_quantiles, calculate_global_quantities
from pyAcousticsAnalysis.plotting.latex_functions import create_latex, merge_pdfs

%load_ext autoreload
%autoreload 2

### Folders

In [4]:
base_folder = "../../formificio_Padova"

In [5]:
foto_folder =  os.path.join(base_folder, "foto").replace("\\", "/")
logo_folder =  os.path.join(base_folder, "logo").replace("\\", "/")
reports_folder =  os.path.join(base_folder, "reports").replace("\\", "/")
preprocessed_folder = os.path.join(reports_folder, "preprocessed").replace("\\", "/")
data_folder = os.path.join(reports_folder, "data").replace("\\", "/")

plots_folder = os.path.join(reports_folder, "plots").replace("\\", "/")
os.makedirs(plots_folder, exist_ok=True) 

In [47]:
with open(os.path.join(data_folder, "metadata.json"), 'r') as f:
    metadata = json.load(f)

filenames = sorted([x for x in os.listdir(preprocessed_folder) if x.endswith(".xlsx")])

out_list = []

for filename in filenames[:]:
    print(filename)
    nome_misura = filename.split("-")[-1].replace(".xlsx", ".s")
    file_path = os.path.join(preprocessed_folder, filename)
    
#    df = read_LD_831_data(file_path)
    df = read_preprocessed_data(file_path)
        
    #plot
    plot_SPL(nome_misura, filename, df, plots_folder, show = False)
    plot_time_history(nome_misura, filename, df, plots_folder, find_impulses = False, show = False)    
    plot_min_SPL(nome_misura, filename, df, plots_folder, show = False)
    quantiles = calculate_quantiles(df)
    global_quantities = calculate_global_quantities(df)
#    print(quantiles)
    
    out_dict = {"filename": filename, 
                "fields": 
                    {
                    "-logo-": logo_folder + "/" + os.listdir(logo_folder)[0][:-4], 
                    "-nome!misura-": nome_misura.replace("_", "\_"), 
                   "-data!ora-": str(df.loc[0, 'Data']),
                    "-SPL!plot-": plots_folder + "/" + filename[:-5] + "_SPL",                       
#                    "-time!history!plot-": "../plots/" + filename[:-5] + "_time_history",
                    "-time!history!plot-":  plots_folder + "/" +filename[:-5] + "_time_history",
                    "-SPL!min!plot-":  plots_folder + "/" + filename[:-5] + "_SPL_min"
                    }
               }    
    
    #add quantiles
    out_dict['fields'].update(quantiles)

    #add global_quantities
    out_dict['fields'].update(global_quantities)
    
    #add metadata    
    metadata_misura = [x for x in metadata if x['filename'] == out_dict['filename']][0]
    out_dict['fields'].update({k: v for k, v in metadata_misura.items() if k != "filename"})

    #update foto path
    out_dict['fields']['-foto-'] = foto_folder + "/" + out_dict['fields']['-foto-']

    #append results
    out_list.append(out_dict)

#    print(df[df['masked'] == True].shape)
    
with open(os.path.join(data_folder, "latex_data.json"), 'w') as f:
    json.dump(out_list, f)

print("end")

<>:28: SyntaxWarning: invalid escape sequence '\_'
<>:28: SyntaxWarning: invalid escape sequence '\_'
/tmp/ipykernel_12784/1872488630.py:28: SyntaxWarning: invalid escape sequence '\_'
  "-nome!misura-": nome_misura.replace("_", "\_"),


831_0003315-20240314 164649-447TH_M.008.xlsx
831_0003315-20240314 170303-447TH_M.009.xlsx
831_0003315-20240314 171901-447TH_M.010.xlsx
831_0003315-20240314 173756-447TH_M.012.xlsx
831_0003315-20240314 175343-447TH_M.013.xlsx
831_0003315-20240314 181055-447TH_M.014.xlsx
831_0003315-20240314 182706-447TH_M.015.xlsx
end


### Create latex

In [48]:
templates_folder = os.path.join("../" "assets", "latex_templates", "ambientale_447")

In [49]:
create_latex(templates_folder, reports_folder)

creating latex for: 831_0003315-20240314 164649-447TH_M.008.xlsx
creating latex for: 831_0003315-20240314 170303-447TH_M.009.xlsx
creating latex for: 831_0003315-20240314 171901-447TH_M.010.xlsx
creating latex for: 831_0003315-20240314 173756-447TH_M.012.xlsx
creating latex for: 831_0003315-20240314 175343-447TH_M.013.xlsx
creating latex for: 831_0003315-20240314 181055-447TH_M.014.xlsx
creating latex for: 831_0003315-20240314 182706-447TH_M.015.xlsx
- all latex files created


In [51]:
merge_pdfs(reports_folder, out_filename = "misure.pdf")

merging 831_0003315-20240314 170303-447TH_M.009.pdf
merging 831_0003315-20240314 173756-447TH_M.012.pdf
merging 831_0003315-20240314 182706-447TH_M.015.pdf
merging 831_0003315-20240314 171901-447TH_M.010.pdf
merging 831_0003315-20240314 164649-447TH_M.008.pdf
merging 831_0003315-20240314 175343-447TH_M.013.pdf
merging 831_0003315-20240314 181055-447TH_M.014.pdf
- all pdf merged
